### Create maps in dense freesurfer space

In [1]:
from pathlib import Path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
from nibabel.freesurfer.io import read_geometry, read_label
from fmri_tools.surface.filter import LaplacianGaussian
from fmri_tools.surface.mesh import Mesh
from fmri_tools.io.surf import (
    read_mgh, write_mgh, write_label, patch_as_mesh, mgh_to_patch, curv_to_patch, 
    label_to_patch
    )
from src.config import DIR_BASE, SESSION
from src.helper import get_composed_label

SUBJ = "p2"
HEMI = "lh"
LAYER = 5
DIR_OUT = "/data/pt_01880/zzz_new"

# make output folders
DIR_DENSE = Path(DIR_OUT) / "dense"
DIR_DENSE.mkdir(parents=True, exist_ok=True)

In [2]:
# get final transformation label to dense freesurfer mesh
file_out = DIR_DENSE / f"{HEMI}.composed.label"
file_in = Path(DIR_BASE) / SUBJ / "anatomy/dense_refined" / f"{HEMI}.white_match_final"
file_in2 = Path(DIR_BASE) / SUBJ / "anatomy/gbb" / f"{HEMI}.white_def2_refined"
file_ind2 = Path(DIR_BASE) / SUBJ / "anatomy/gbb" / f"{HEMI}.white_def2_refined_ind.txt"
file_ind3 = Path(DIR_BASE) / SUBJ / "anatomy/dense_epi" / f"{HEMI}.white_def2_ind"
get_composed_label(file_out, file_in, file_in2, file_ind2, file_ind3)

100%|██████████| 280685/280685 [13:10<00:00, 354.85it/s]


In [3]:
# transform arrays
file_ind = DIR_DENSE / f"{HEMI}.composed.label"
file_geom = Path(DIR_BASE) / SUBJ / "anatomy/dense" / f"{HEMI}.white"
file_mgh = []
for sess in SESSION[SUBJ]:
    for day in SESSION[SUBJ][sess]:
        if "_uncorrected" not in sess:
            file_mgh.append(
                Path(DIR_BASE) 
                / SUBJ 
                / f"odc/results/Z/sampled/Z_all_left_right_{sess}{day}" 
                / f"{HEMI}.Z_all_left_right_{sess}{day}_layer_{LAYER}.mgh"
            )

ind = read_label(file_ind)
vtx, _ = read_geometry(file_geom)
for f in file_mgh:
    file_out = DIR_DENSE / f"{Path(f).stem}_dense.mgh"
    arr, _, _ = read_mgh(f)
    res = np.zeros(len(vtx))
    res[ind] = arr
    write_mgh(file_out, res)

/data/pt_01880/source/miniconda3/envs/odc/lib/python3.10/site-packages/nibabel/spatialimages.py:495: UserWarning: Using large vector Freesurfer hack; header will not be compatible with SPM or FSL
  hdr.set_data_shape(shape)


In [4]:
# transform label
file_ind = DIR_DENSE / f"{HEMI}.composed.label"
file_geom = Path(DIR_BASE) / SUBJ / "/data/pt_01880/Experiment1_ODC/p1/anatomy/dense/lh.white"
file_label = []
for label in ["fov", "v1", "v2", "v2a", "v2b", "v3", "v3a", "v3b"]:
    file_label.append(Path(DIR_BASE) / SUBJ / "anatomy/label" / f"{HEMI}.{label}.label")

ind = read_label(file_ind)
for f in file_label:
    file_out = DIR_DENSE / f"{Path(f).stem}_dense.label"
    l_ = read_label(f)
    res = ind[l_]
    write_label(file_out, res)

In [5]:
# get average maps
for sess in SESSION[SUBJ]:
    days = SESSION[SUBJ][sess]
    if "_uncorrected" not in sess:
        file_mgh1 = (
            DIR_DENSE 
            / f"{HEMI}.Z_all_left_right_{sess}{days[0]}_layer_{LAYER}_dense.mgh"
        )
        file_mgh2 = (
            DIR_DENSE
            / f"{HEMI}.Z_all_left_right_{sess}{days[1]}_layer_{LAYER}_dense.mgh"
        )
        file_out = DIR_DENSE / f"{HEMI}.Z_all_left_right_{sess}_layer_{LAYER}_avg.mgh"
        arr1, _, _ = read_mgh(file_mgh1)
        arr2, _, _ = read_mgh(file_mgh2)
        res = (arr1 + arr2) / 2
        write_mgh(file_out, res)